# Measurement of an Impulse Response with a Microphone

This notebook scetches how to measure impulse responses with a single mircophone. 

* Two sweeps are played, one for the loudspeaker and one for the analog feedback. 
* Two channels are recorded, the microphone and the analog feedback.

Data are stored in a subfolder 'data', which needs to be present.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.fft import rfft as fft, rfftfreq as fftfreq
import soundfile as sf
import time
import scipy.io as sio

from IPython import display

import acoustics_hardware

# Set up hardware

In [ ]:
print(acoustics_hardware.devices.AudioDevice.get_devices())

name = 'Orion 32'
fs = 48000
device = acoustics_hardware.devices.AudioDevice(name=name, fs=fs)

# 2 inputs (0..7)
device.add_input(0)
device.add_input(1)

# 2 outputs (0..7)
device.add_output(0)
device.add_output(1)

amplitude_db = -20
sweep_duration = 3 # sec

amplitude_lin = 10**(amplitude_db/20)

# Signal is automatically played through the first 2 channels
generator = acoustics_hardware.generators.SweepGenerator(device=device,
    start_frequency=50, stop_frequency=20e3, duration=sweep_duration, repetitions=1, amplitude=amplitude_lin)

# Define functions for plotting (for testing only)

In [ ]:
# Define the functions for plotting
def plot_data_t(data):
    #data.shape
    channels = np.arange(data.shape[0])

    for channel in channels:
        #plt.plot(data[2,:], 'b', data[0,:],'r',data[1,:], 'g')
        plt.plot(data[channel,:]) 
        plt.xlabel("t (taps)")
        plt.show()
        
def plot_data_f(data, fs):
    freq_data = fft(data)
    f = fftfreq(data.shape[1], 1/fs)
    
    channels = np.arange(data.shape[0])
    
    for channel in channels:
        plt.semilogx(f, 20*np.log10(abs(freq_data[channel-1,:])).T)
        plt.xlim(30, fs/2)
        plt.xlabel("f (Hz)")
        plt.show() 

# Perform measurement

In [ ]:
file_name = 'recorded_data.mat'

device.start()

# Launch the measurement
generator.reset()
device.input_active.set()
time.sleep(.1)
device.output_active.set()

# Wait for the measurement to happen
time.sleep(sweep_duration + 3)

# Stop the recording
device.stop()

# Retreive the recorded data
data = device.get_input_data()

# Store the data
sio.savemat(file_name, {'recorded_signal': np.transpose(data), 'fs': fs})

# Play audio
display.display(
    display.Markdown('### Channel 1'),
    display.Audio(data[0,:], rate=fs),
    display.Markdown('### Channel 2'),
    display.Audio(data[1,:], rate=fs))

plot_data_t(data)
plot_data_f(data, fs) 